In [0]:
# Prepare for realtime tensorboard
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!ls
!unzip ngrok-stable-linux-amd64.zip

In [0]:
!pip install tf-nightly
!ls
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
# Prepare for dataset
!wget https://storage.googleapis.com/plantdata/PlantVillage.zip
!ls
!unzip PlantVillage.zip 

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import json
import datetime
import os

TRAIN_DIR   = "./PlantVillage" + "/train"
VALID_DIR   = "./PlantVillage" + "/validation"
MODEL_DIR   = "/gdrive/My Drive/DA/models/"
IMAGE_SIZE  = (299, 299)
BATCH_SIZE  = 64
LEARNING_RATE = 0.0001

with open('/gdrive/My Drive/DA/Dataset/PlantVillage/categories.json', 'r') as f:
    cat_to_name = json.load(f)
    classes = list(cat_to_name.values())

print (classes)

In [0]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range=0,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0, 
    height_shift_range=0,
    shear_range=0.1,
    zoom_range=0.0,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, 
    subset="training", 
    shuffle=True, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)


In [0]:
model = tf.keras.Sequential([
    tf.keras.applications.InceptionV3(input_shape=IMAGE_SIZE+(3,),  include_top=False, weights='imagenet'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax',
                           kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])

latest = tf.train.latest_checkpoint(MODEL_DIR)
if latest:
    print("*"*50)
    print("Load weight from last")
    print(latest)
    model.load_weights(latest)
    print("*"*50)

model.compile(
   optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE), 
   loss='categorical_crossentropy',
   metrics=['accuracy'])

model.summary()


In [0]:
now = datetime.datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

if not os.path.isdir(LOG_DIR):
    os.mkdir(LOG_DIR)

log_dir = LOG_DIR
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, update_freq='batch',profile_batch=0)

if not os.path.isdir(MODEL_DIR):
    os.mkdir(MODEL_DIR)

checkpoint_path = MODEL_DIR + "/cp-" + now + "-{epoch:04d}.ckpt"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    save_freq='epoch')

EPOCHS=10 #@param {type:"integer"}
STEPS_EPOCHS = train_generator.samples//train_generator.batch_size

history = model.fit_generator( 
          train_generator,
          steps_per_epoch=STEPS_EPOCHS,
          epochs=EPOCHS,
          callbacks=[tensorboard_callback, checkpoint_callback])

In [0]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    horizontal_flip=True,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    shear_range=0.2, 
    zoom_range=0.2,
    fill_mode='nearest')

test_generator = test_datagen.flow_from_directory(
    VALID_DIR, 
    subset="training", 
    shuffle=True, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

In [0]:
results = model.evaluate_generator(test_generator)
print('test loss, test acc:', results)